# vol-statement-to-item-callnumber

**The purpose of this notebook / script is to append the volume statement to the end of the call number associated with the item record.**

Example:

    Item Record Number: 12382839
    Title: Sports Illustrated
    Item Call Number: periodical
    Volume Statement (associated with the volume record): V. 135 NO. 2 Mar 2024

After the change, the *Item Call Number* becomes: **periodical V. 135 NO. 2 Mar 2024**.

---

**NOTE**: The Sierra API endpoint for item -- `GET items/{id}` -- appears to combine both the call number and volume statement into one string.

e.g. item record number `12382839` has an attached volume with the information above, returns as the `callNumber` value in the response the following:

    Periodical      V. 135 NO. 2 Mar 2024




In [10]:
%%capture
!pip install sierra-ils-utils  # install from PyPI

import json

with open('/home/jovyan/.config/sierra/api-vol-stmt-to-call-number.json') as f:
    sierra_config = json.load(f)
    
api_key = sierra_config.get('vol-stmt-to-call-number-key')
api_secret = sierra_config.get('vol-stmt-to-call-number-secret')

# api_key = userdata.get('vol-stmt-to-call-number-key')
# api_secret = userdata.get('vol-stmt-to-call-number-secret')

import sierra_ils_utils

base_url = 'https://classic.cincinnatilibrary.org/iii/sierra-api/v6/'

 # create the API client
sierra_api = sierra_ils_utils.SierraAPI(
  base_url,   # Base URL of the Sierra API
  api_key,    # Sierra API Key
  api_secret  # Sierra API Secret
)

In [17]:
# send a normal request to set the header / auth
response = sierra_api.get('info/token')
print(response.raw_response.status_code)

# the sierra-ils-utils library exposes an HTTPX Client
# https://www.python-httpx.org/advanced/clients/
# type(sierra_api.session)  # httpx.Client

filename = './export (3).txt'

import csv

with open(filename) as f:
    record_data = list(csv.DictReader(f))
    
print(len(record_data), record_data[0].keys())

200
74 dict_keys(['RECORD #(VOLUME)'])


In [18]:
# NOTE:
# Use this cell if the input file containes volume records -- e.g. `RECORD #(VOLUME)`

#  the record number in the data has the record number type, and the check-digit
# ... we can remove them. e.g. 'i101150647' becomes '10115064'
vol_record_numbers = [
    str(record['RECORD #(VOLUME)'][1:-1])  # remove the first and last characters
    for record
    in record_data
]

print(f'Processing {len(vol_record_numbers)} VOLUME records')

all_item_records = []

# resolve the volume records to item records
for i, vol_record in enumerate(vol_record_numbers):
    response = sierra_api.session.get(
        base_url + f'volumes/{vol_record}',
        params={'fields': 'items'}
    )
    
    response_items = [item.split('/')[-1] for item in response.json().get('items')]
    # print(response_items)
    all_item_records.extend(response_items)  # Add the items to the big list
    
item_record_numbers = list(set(all_item_records))  # just for sanity, dedupe and return the list

print(f'Processing {len(item_record_numbers)} ITEM records')

Processing 74 VOLUME records
Processing 392 ITEM records


In [ ]:
# NOTE:
# OR ...
# Use this cell if the input file containes item records -- e.g. `RECORD #(VOLUME)`

# import the text/csv file -- it should have 3 columns ...
# (bib record number, item record number, and volume statement)
filename = './export (3).txt'

import csv

with open(filename) as f:
  record_data = list(csv.DictReader(f))

#  the record number in the data has the record number type, and the check-digit
# ... we can remove them. e.g. 'i101150647' becomes '10115064'
item_record_numbers = [
    str(record['RECORD #(ITEM)'][1:-1])  # remove the first and last characters
    for record
    in record_data
]

print(f'Processing {len(item_record_numbers)} ITEM records')

In [22]:
from sierra_ils_utils import Models as sierra_models

# record all the barcodes
barcodes = []

for i, id in enumerate(item_record_numbers):
    print(f'Processing record {i+1} of {len(item_record_numbers)}')
    response = sierra_api.get(
        'items/{id}',
        path_params={'id': id},
        params={'fields': 'barcode,callNumber,volumes'}
    )
    print(response.data.id, response.data.barcode, response.data.callNumber, response.data.volumes)
    barcodes.append(response.data.barcode)

    # create an Item patch object
    patched_item = sierra_models.Item(
        id=response.data.id,
        callNumber=response.data.callNumber  # simply use the call number returned ...
                                             # from the original request (see note above)
    )

    # find out if the call number contains the volume statment already (will appear 2 times)
    #
    # extract the volume record number
    volume_record_num = response.data.volumes[0].split('/')[-1]
    # request the volume record data
    response_vol = sierra_api.session.get(base_url + f'volumes/{volume_record_num}')
    # Safely extract the volume from the response
    volume = response_vol.json().get('volume')

    if volume and patched_item.callNumber.count(response_vol.json()['volume']) >= 2:
        # if the call number contains the volume statement exactly 2x, then we
        # don't need to process it again -- it's already done
        print('already processed')

    else:
        # Update an item record
        response = sierra_api.session.put(
            base_url + f'items/{patched_item.id}',
            json={
                'callNumbers': [patched_item.callNumber]
            }
        )

        print(response.status_code)
        print(response.text)

Processing record 1 of 392
11625262 A000069385674 Periodical      V. 53 NO. 1 May/Jun 2022 V. 53 NO. 1 May/Jun 2022 ['https://classic.cincinnatilibrary.org/iii/sierra-api/v6/volumes/1521594']
already processed
Processing record 2 of 392
9831239 A000051261725 Periodical V. 48 NO. 11 Apr 2018 V. 48 NO. 11 Apr 2018 ['https://classic.cincinnatilibrary.org/iii/sierra-api/v6/volumes/1481306']
already processed
Processing record 3 of 392
9729239 A000051237428 Periodical V. 48 NO. 9 Feb 2018 V. 48 NO. 9 Feb 2018 ['https://classic.cincinnatilibrary.org/iii/sierra-api/v6/volumes/1481303']
already processed
Processing record 4 of 392
11560710 A000069343293 Periodical V. 52 NO. 6 Mar/Apr 2022 V. 52 NO. 6 Mar/Apr 2022 ['https://classic.cincinnatilibrary.org/iii/sierra-api/v6/volumes/1521593']
already processed
Processing record 5 of 392
11857232 A000045522505 Periodical      V. 102 NO. 43 Oct 24, 2022 V. 102 NO. 43 Oct 24, 2022 ['https://classic.cincinnatilibrary.org/iii/sierra-api/v6/volumes/15290

In [23]:
import csv
from datetime import datetime

# Get the current date in YYYY-MM-DD format
current_date = datetime.now().strftime('%Y-%m-%d')

# Automatically name the output file with the current date
output_file = f'barcodes_{current_date}.csv'

# Write the barcodes to a CSV file with no header
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    for barcode in barcodes:
        writer.writerow([barcode])

print(f'Barcodes written to {output_file}')

Barcodes written to barcodes_2024-08-16.csv
